In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np

class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, 100)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.summary()

        img = Input(shape=self.img_shape)

        # Extract feature representation
        features = model(img)

        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes+1, activation="softmax")(features)

        return Model(img, [validity, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # The labels of the digits that the generator tries to create an
            # image representation of
            sampled_labels = np.random.randint(0, 10, (batch_size, 1))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, sampled_labels])

            # Image labels. 0-9 if image is valid or 10 if it is generated (fake)
            img_labels = y_train[idx]
            fake_labels = 10 * np.ones(img_labels.shape)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, [valid, img_labels])
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, fake_labels])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.save_model()
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model/%s.json" % model_name
            weights_path = "saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator")
        save(self.discriminator, "discriminator")


if __name__ == '__main__':
    acgan = ACGAN()
    acgan.train(epochs=14000, batch_size=32, sample_interval=200)


C:\Users\adward\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 32)          0         
__________

C:\Users\adward\Anaconda3\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 3.401515, acc.: 31.25%, op_acc: 7.81%] [G loss: 3.519892]
1 [D loss: 3.716880, acc.: 40.62%, op_acc: 1.56%] [G loss: 3.127439]
2 [D loss: 3.468428, acc.: 40.62%, op_acc: 9.38%] [G loss: 3.382676]
3 [D loss: 3.321783, acc.: 54.69%, op_acc: 10.94%] [G loss: 3.446068]
4 [D loss: 2.971587, acc.: 51.56%, op_acc: 32.81%] [G loss: 3.268886]
5 [D loss: 2.996466, acc.: 51.56%, op_acc: 28.12%] [G loss: 3.554688]
6 [D loss: 2.821564, acc.: 50.00%, op_acc: 37.50%] [G loss: 3.623351]
7 [D loss: 2.904175, acc.: 46.88%, op_acc: 34.38%] [G loss: 3.364041]
8 [D loss: 2.599151, acc.: 54.69%, op_acc: 43.75%] [G loss: 3.656256]
9 [D loss: 2.642506, acc.: 57.81%, op_acc: 43.75%] [G loss: 3.784691]
10 [D loss: 2.669879, acc.: 56.25%, op_acc: 48.44%] [G loss: 3.763494]
11 [D loss: 2.486629, acc.: 56.25%, op_acc: 43.75%] [G loss: 4.102346]
12 [D loss: 2.735775, acc.: 57.81%, op_acc: 46.88%] [G loss: 4.123521]
13 [D loss: 2.266556, acc.: 65.62%, op_acc: 50.00%] [G loss: 4.216159]
14 [D loss: 2.24142

116 [D loss: 2.163028, acc.: 40.62%, op_acc: 54.69%] [G loss: 4.192869]
117 [D loss: 1.934883, acc.: 43.75%, op_acc: 60.94%] [G loss: 5.257723]
118 [D loss: 1.982613, acc.: 50.00%, op_acc: 60.94%] [G loss: 4.931164]
119 [D loss: 1.650808, acc.: 68.75%, op_acc: 67.19%] [G loss: 4.764508]
120 [D loss: 1.923552, acc.: 50.00%, op_acc: 60.94%] [G loss: 5.206688]
121 [D loss: 1.941937, acc.: 59.38%, op_acc: 62.50%] [G loss: 5.402947]
122 [D loss: 1.926826, acc.: 51.56%, op_acc: 57.81%] [G loss: 5.616865]
123 [D loss: 1.874076, acc.: 64.06%, op_acc: 59.38%] [G loss: 5.313206]
124 [D loss: 1.808631, acc.: 54.69%, op_acc: 60.94%] [G loss: 5.587356]
125 [D loss: 1.653377, acc.: 64.06%, op_acc: 71.88%] [G loss: 5.509132]
126 [D loss: 1.919999, acc.: 53.12%, op_acc: 57.81%] [G loss: 5.214029]
127 [D loss: 1.807137, acc.: 53.12%, op_acc: 68.75%] [G loss: 5.271622]
128 [D loss: 1.778671, acc.: 54.69%, op_acc: 60.94%] [G loss: 5.002757]
129 [D loss: 1.940506, acc.: 62.50%, op_acc: 62.50%] [G loss: 5.

...

13508 [D loss: 1.413844, acc.: 60.94%, op_acc: 59.38%] [G loss: 1.768935]
13509 [D loss: 1.497812, acc.: 48.44%, op_acc: 46.88%] [G loss: 1.736535]
13510 [D loss: 1.534292, acc.: 40.62%, op_acc: 51.56%] [G loss: 1.695674]
13511 [D loss: 1.389767, acc.: 50.00%, op_acc: 57.81%] [G loss: 1.752768]
13512 [D loss: 1.534024, acc.: 53.12%, op_acc: 39.06%] [G loss: 1.676944]
13513 [D loss: 1.517771, acc.: 53.12%, op_acc: 48.44%] [G loss: 1.651201]
13514 [D loss: 1.458887, acc.: 60.94%, op_acc: 51.56%] [G loss: 1.702419]
13515 [D loss: 1.426724, acc.: 62.50%, op_acc: 53.12%] [G loss: 1.663260]
13516 [D loss: 1.555618, acc.: 48.44%, op_acc: 42.19%] [G loss: 1.563704]
13517 [D loss: 1.482718, acc.: 51.56%, op_acc: 54.69%] [G loss: 1.770303]
13518 [D loss: 1.552191, acc.: 54.69%, op_acc: 50.00%] [G loss: 1.666233]
13519 [D loss: 1.538779, acc.: 46.88%, op_acc: 54.69%] [G loss: 1.637064]
13520 [D loss: 1.562690, acc.: 40.62%, op_acc: 45.31%] [G loss: 1.725507]
13521 [D loss: 1.604281, acc.: 46.88%,

13619 [D loss: 1.512985, acc.: 45.31%, op_acc: 46.88%] [G loss: 1.694422]
13620 [D loss: 1.476475, acc.: 64.06%, op_acc: 46.88%] [G loss: 1.684436]
13621 [D loss: 1.483004, acc.: 56.25%, op_acc: 48.44%] [G loss: 1.641396]
13622 [D loss: 1.516202, acc.: 50.00%, op_acc: 45.31%] [G loss: 1.689593]
13623 [D loss: 1.370769, acc.: 70.31%, op_acc: 53.12%] [G loss: 1.795897]
13624 [D loss: 1.521922, acc.: 51.56%, op_acc: 48.44%] [G loss: 1.689642]
13625 [D loss: 1.402460, acc.: 53.12%, op_acc: 54.69%] [G loss: 1.623939]
13626 [D loss: 1.409487, acc.: 59.38%, op_acc: 54.69%] [G loss: 1.716847]
13627 [D loss: 1.553786, acc.: 43.75%, op_acc: 42.19%] [G loss: 1.821001]
13628 [D loss: 1.790662, acc.: 35.94%, op_acc: 32.81%] [G loss: 1.564738]
13629 [D loss: 1.476495, acc.: 56.25%, op_acc: 43.75%] [G loss: 1.644498]
13630 [D loss: 1.455332, acc.: 51.56%, op_acc: 51.56%] [G loss: 1.557843]
13631 [D loss: 1.493440, acc.: 48.44%, op_acc: 48.44%] [G loss: 1.607344]
13632 [D loss: 1.608237, acc.: 43.75%,

13730 [D loss: 1.525356, acc.: 56.25%, op_acc: 50.00%] [G loss: 1.593791]
13731 [D loss: 1.492633, acc.: 62.50%, op_acc: 42.19%] [G loss: 1.686794]
13732 [D loss: 1.569160, acc.: 59.38%, op_acc: 50.00%] [G loss: 1.567775]
13733 [D loss: 1.439619, acc.: 60.94%, op_acc: 56.25%] [G loss: 1.481830]
13734 [D loss: 1.546144, acc.: 46.88%, op_acc: 40.62%] [G loss: 1.511127]
13735 [D loss: 1.538524, acc.: 53.12%, op_acc: 51.56%] [G loss: 1.573629]
13736 [D loss: 1.437971, acc.: 54.69%, op_acc: 62.50%] [G loss: 1.462929]
13737 [D loss: 1.432065, acc.: 54.69%, op_acc: 53.12%] [G loss: 1.691622]
13738 [D loss: 1.572951, acc.: 51.56%, op_acc: 39.06%] [G loss: 1.675073]
13739 [D loss: 1.392471, acc.: 62.50%, op_acc: 50.00%] [G loss: 1.628340]
13740 [D loss: 1.590553, acc.: 43.75%, op_acc: 43.75%] [G loss: 1.500024]
13741 [D loss: 1.519596, acc.: 46.88%, op_acc: 40.62%] [G loss: 1.609940]
13742 [D loss: 1.405634, acc.: 46.88%, op_acc: 56.25%] [G loss: 1.451829]
13743 [D loss: 1.556523, acc.: 60.94%,

13841 [D loss: 1.574337, acc.: 50.00%, op_acc: 45.31%] [G loss: 1.608031]
13842 [D loss: 1.514355, acc.: 53.12%, op_acc: 51.56%] [G loss: 1.672061]
13843 [D loss: 1.291991, acc.: 73.44%, op_acc: 60.94%] [G loss: 1.632408]
13844 [D loss: 1.495909, acc.: 50.00%, op_acc: 51.56%] [G loss: 1.477881]
13845 [D loss: 1.545619, acc.: 48.44%, op_acc: 34.38%] [G loss: 1.717572]
13846 [D loss: 1.392941, acc.: 53.12%, op_acc: 54.69%] [G loss: 1.591293]
13847 [D loss: 1.327959, acc.: 71.88%, op_acc: 67.19%] [G loss: 1.683462]
13848 [D loss: 1.508445, acc.: 43.75%, op_acc: 43.75%] [G loss: 1.501820]
13849 [D loss: 1.519511, acc.: 48.44%, op_acc: 42.19%] [G loss: 1.580792]
13850 [D loss: 1.360609, acc.: 59.38%, op_acc: 62.50%] [G loss: 1.544163]
13851 [D loss: 1.463101, acc.: 48.44%, op_acc: 45.31%] [G loss: 1.594683]
13852 [D loss: 1.528343, acc.: 51.56%, op_acc: 46.88%] [G loss: 1.527369]
13853 [D loss: 1.491652, acc.: 46.88%, op_acc: 50.00%] [G loss: 1.507142]
13854 [D loss: 1.542372, acc.: 48.44%,

13952 [D loss: 1.371619, acc.: 73.44%, op_acc: 57.81%] [G loss: 1.650122]
13953 [D loss: 1.539676, acc.: 39.06%, op_acc: 39.06%] [G loss: 1.557848]
13954 [D loss: 1.502274, acc.: 53.12%, op_acc: 50.00%] [G loss: 1.611698]
13955 [D loss: 1.490586, acc.: 59.38%, op_acc: 57.81%] [G loss: 1.575873]
13956 [D loss: 1.457851, acc.: 46.88%, op_acc: 54.69%] [G loss: 1.874017]
13957 [D loss: 1.275430, acc.: 59.38%, op_acc: 71.88%] [G loss: 1.759660]
13958 [D loss: 1.482134, acc.: 59.38%, op_acc: 42.19%] [G loss: 1.610966]
13959 [D loss: 1.592139, acc.: 37.50%, op_acc: 40.62%] [G loss: 1.558367]
13960 [D loss: 1.626253, acc.: 39.06%, op_acc: 34.38%] [G loss: 1.520624]
13961 [D loss: 1.424571, acc.: 50.00%, op_acc: 50.00%] [G loss: 1.662614]
13962 [D loss: 1.449815, acc.: 43.75%, op_acc: 59.38%] [G loss: 1.631351]
13963 [D loss: 1.692640, acc.: 37.50%, op_acc: 35.94%] [G loss: 1.538415]
13964 [D loss: 1.797766, acc.: 43.75%, op_acc: 37.50%] [G loss: 1.604581]
13965 [D loss: 1.637372, acc.: 54.69%,